In [3]:
import numpy as np
import pandas as pd
import pywt

In [240]:
df = pd.read_csv('Ejemplos/Distribuciones/Binomial-fin.csv',index_col='indice')

In [239]:
mult = pd.read_csv('Ejemplos/MULTIVARIABLE/ARMA-TEND-PER-fin.csv',index_col='indice')

In [6]:
def series_periodos(inicio, periodos, freq): 
    serie = pd.date_range(start=inicio, periods=periodos, freq=freq)
    return serie

TRASLACION

In [ ]:
# Desplazamiento espacial de la serie

def traslacion(df,shift,freq):
    
    df_trasl =df.copy()
    for x in df_trasl.columns:
        data = df[x]
        data_augmented = df[x] + shift
        datos = np.concatenate((data.values,data_augmented))
        if x == df.columns[0]:
            indice = series_periodos(df.index[0],len(datos),freq)
            df_trasl = pd.DataFrame(data=datos,index=indice,columns=[x])
        else:
            df_new = pd.DataFrame(data = datos,index=indice,columns=[x])
            df_trasl = df_trasl.join(df_new, how="outer")
    return df_trasl

In [122]:
traslacion(df,10,'M')

,Valor
indice,
2014-01-31,63
2014-02-28,62
2014-03-31,61
2014-04-30,62
2014-05-31,60
...,...
2023-08-31,61
2023-09-30,60
2023-10-31,60


In [123]:
traslacion(mult,10,'M')

,Altura,Dinero,Peso
indice,,,
2014-01-31,80.555312,139.091399,122.902687
2014-02-28,78.346558,123.850549,112.844928
2014-03-31,80.871320,97.866499,121.120237
2014-04-30,79.665688,83.274410,118.697997
2014-05-31,97.544611,69.460597,120.748102
...,...,...,...
2023-08-31,103.631116,10.000000,118.131639
2023-09-30,102.122000,10.000000,122.406146
2023-10-31,101.822198,10.000000,119.946900


ESCALADO

In [128]:
def scale(series,factor):
    return series * factor

In [132]:
# Multiplicación por un factor de la serie

def escalado(df,freq,factor):
    
    df_esc =df.copy()
    for x in df_esc.columns:
        data = df[x]
        data_augmented = df[x]*factor
        datos = np.concatenate((data.values,data_augmented))
        if x == df.columns[0]:
            indice = series_periodos(df.index[0],len(datos),freq)
            df_esc = pd.DataFrame(data=datos,index=indice,columns=[x])
        else:
            df_new = pd.DataFrame(data = datos,index=indice,columns=[x])
            df_esc= df_esc.join(df_new, how="outer")
    return df_esc

In [133]:
df_scale(df,'M',0.45)

,Valor
indice,
2014-01-31,23.85
2014-02-28,23.40
2014-03-31,22.95
2014-04-30,23.40
2014-05-31,22.50
...,...
2023-08-31,22.95
2023-09-30,22.50
2023-10-31,22.50


In [134]:
df_scale(mult,'M',0.7)

,Altura,Dinero,Peso
indice,,,
2014-01-31,49.388718,9.036398e+01,79.031881
2014-02-28,47.842591,7.969538e+01,71.991450
2014-03-31,49.609924,6.150655e+01,77.784166
2014-04-30,48.765982,5.129209e+01,76.088598
2014-05-31,61.281227,4.162242e+01,77.523672
...,...,...,...
2023-08-31,65.541781,1.007777e-08,75.692147
2023-09-30,64.485400,7.289554e-09,78.684302
2023-10-31,64.275539,6.576515e-09,76.962830


INTERPOLACION:

In [23]:
from scipy.interpolate import interp1d
from scipy.interpolate import UnivariateSpline, CubicSpline

In [167]:
# Definimos nuevos datos indicando el número de datos a generar, la frequencia y el tipo de interpolación

def interpolacion_min_max(df,kind,num,freq):
    
    df=df.reset_index()
    indices=df.index.values
    indice=series_periodos(df[df.columns[0]][0],num+df.shape[0],freq)
    x = indices 
    for i in range(1,len(df.columns)):
        y = df[df.columns[i]]
        inicio = min(df[df.columns[i]].argmin(),df[df.columns[i]].argmax())
        fin = max(df[df.columns[i]].argmin(),df[df.columns[i]].argmax())
        f = interp1d(x, y, kind=kind) # kind ='linear' / 'cubic' / 'quadratic'
        x_new = np.linspace(inicio,fin, num=num)  # New x values
        y_new = f(x_new)  # Interpolated y values
        if i==1:
            df_int = pd.DataFrame(data=np.concatenate((y.values.reshape(-1),y_new)),index=indice,columns=[df.columns[i]])
        else :     
            df_n = pd.DataFrame(data=np.concatenate((y.values.reshape(-1),y_new)),index=indice,columns=[df.columns[i]])
            df_int= df_int.join(df_n, how="outer")
            
    return df_int

In [165]:
# Definimos nuevos datos indicando el número de datos a generar, la frequencia y el tipo de interpolación (lineal/cubico).

def interpolacion_normal(df,kind,num,freq):
    
    df=df.reset_index()
    indices=df.index.values
    indice=series_periodos(df[df.columns[0]][0],num+df.shape[0],freq)
    x = indices 
    for i in range(1,len(df.columns)):
        y = df[df.columns[i]]
        f = interp1d(x, y, kind=kind) # kind = 'linear' / 'cubic' / 'quadratic'
        x_new = np.linspace(0,df.shape[0]-1, num=num)  # New x values
        y_new = f(x_new)  # Interpolated y values
        if i==1:
            df_int = pd.DataFrame(data=np.concatenate((y.values.reshape(-1),y_new)),index=indice,columns=[df.columns[i]])
        else :     
            df_n = pd.DataFrame(data=np.concatenate((y.values.reshape(-1),y_new)),index=indice,columns=[df.columns[i]])
            df_int= df_int.join(df_n, how="outer")
            
    return df_int

In [168]:
interpolacion_min_max(df,'linear',500,'M')

,Valor
2014-01-31,53.000000
2014-02-28,52.917836
2014-03-31,52.835671
2014-04-30,52.753507
2014-05-31,52.671343
...,...
2055-04-30,52.685371
2055-05-31,53.014028
2055-06-30,53.342685
2055-07-31,53.671343


In [169]:
interpolacion_normal(df,'cubic',140,'M')

,Valor
2014-01-31,53.000000
2014-02-28,53.013821
2014-03-31,53.023910
2014-04-30,53.030379
2014-05-31,53.033338
...,...
2025-04-30,50.239558
2025-05-31,50.176059
2025-06-30,50.114805
2025-07-31,50.056038


In [170]:
interpolacion_normal(mult,'linear',190,'M')

,Altura,Dinero,Peso
2014-01-31,70.555312,1.290914e+02,112.902687
2014-02-28,70.543625,1.194953e+02,112.849472
2014-03-31,70.531939,1.071139e+02,112.796256
2014-04-30,70.520252,9.075362e+01,112.743041
2014-05-31,70.508566,8.030023e+01,112.689825
...,...,...,...
2029-06-30,68.393305,9.923199e-09,103.057791
2029-07-31,68.381618,9.199082e-09,103.004575
2029-08-31,68.369931,8.088760e-09,102.951359
2029-09-30,68.358245,7.114336e-09,102.898144


In [164]:
mult

,Altura,Dinero,Peso
indice,,,
2014-01-31,70.555312,1.290914e+02,112.902687
2014-02-28,68.346558,1.138505e+02,102.844928
2014-03-31,70.871320,8.786650e+01,111.120237
2014-04-30,69.665688,7.327441e+01,108.697997
2014-05-31,87.544611,5.946060e+01,110.748102
...,...,...,...
2023-08-31,93.631116,1.439682e-08,108.131639
2023-09-30,92.122000,1.041365e-08,112.406146
2023-10-31,91.822198,9.395022e-09,109.946900


In [171]:
interpolacion_min_max(mult,'cubic',260,'M')

,Altura,Dinero,Peso
2014-01-31,70.555312,129.091399,112.902687
2014-02-28,67.726644,129.091399,112.020950
2014-03-31,67.562320,129.091399,111.182921
2014-04-30,68.864566,129.091399,110.387758
2014-05-31,70.435608,129.091399,109.634620
...,...,...,...
2035-04-30,93.330341,129.091399,113.835241
2035-05-31,92.785955,129.091399,113.856508
2035-06-30,93.901389,129.091399,113.866641
2035-07-31,95.990630,129.091399,113.865116


In [29]:
def interpolate(data):
    interpolated_data = []
    for i in range(len(data) - 1):
        interpolated_data.append(data[i])
        interpolated_data.append((data[i] + data[i + 1]) / 2)  # Punto intermedio
    interpolated_data.append(data[-1])
    return np.array(interpolated_data)

In [30]:
# Añadimos datos que sean el punto de medio entre dos datos consecutivos

def punto_medio(df,freq):
    
    for x in df.columns:
        data = df[x]
        a = interpolate(data)
        if x == df.columns[0]:
            indice = series_periodos(df.index[0],len(a),freq)
            df_pm = pd.DataFrame(data=a,index=indice,columns=[x])
        else:
            df_new = pd.DataFrame(data=a,index=indice,columns=[x])
            df_pm = df_pm.join(df_new, how="outer")
    return df_pm

In [172]:
punto_medio(df,'M')

,Valor
2014-01-31,53.0
2014-02-28,52.5
2014-03-31,52.0
2014-04-30,51.5
2014-05-31,51.0
...,...
2033-07-31,50.0
2033-08-31,50.5
2033-09-30,51.0
2033-10-31,51.0


In [173]:
punto_medio(mult,'M')

,Altura,Dinero,Peso
2014-01-31,70.555312,1.290914e+02,112.902687
2014-02-28,69.450935,1.214710e+02,107.873808
2014-03-31,68.346558,1.138505e+02,102.844928
2014-04-30,69.608939,1.008585e+02,106.982583
2014-05-31,70.871320,8.786650e+01,111.120237
...,...,...,...
2033-07-31,91.822198,9.395022e-09,109.946900
2033-08-31,92.071548,8.513295e-09,109.699618
2033-09-30,92.320897,7.631569e-09,109.452335
2033-10-31,92.774662,6.933304e-09,108.396959


In [34]:
def spline_interpolation_linear(data, num,s=1):
    x = np.arange(len(data))
    spline = UnivariateSpline(x, data, s=s)
    x_new = np.linspace(0,len(data)-1, num=num)
    return spline(x_new)

def spline_interpolation_cubic(data, num):
    x = np.arange(len(data))
    spline = CubicSpline(x,data)
    x_new = np.linspace(0,len(data)-1, num=num)
    return spline(x_new)

In [174]:
spline_interpolation_linear(df,50,3)

,Valor
indice,
2014-01-31,53.003479
2014-02-28,51.981536
2014-03-31,51.040921
2014-04-30,51.954335
2014-05-31,50.015175
...,...
2023-08-31,50.775734
2023-09-30,49.982316
2023-10-31,50.186066


In [175]:
spline_interpolation_cubic(mult,30,5)

,Altura,Dinero,Peso
indice,,,
2014-01-31,70.542076,129.144591,112.890606
2014-02-28,68.422802,113.502838,102.904763
2014-03-31,70.673322,88.727165,110.994718
2014-04-30,69.980127,72.294407,108.846487
2014-05-31,87.190138,59.804044,110.641111
...,...,...,...
2023-08-31,93.229042,-0.001835,108.175462
2023-09-30,92.316471,-0.002615,112.325472
2023-10-31,91.877603,-0.003461,110.010623


In [ ]:
def interpolacion_spline(df,tipo,num,freq,s):
    
    indice=series_periodos(df.index[0],num+df.shape[0],freq)
    for x in df.columns:
        y=df[x]
        if tipo=='linear': 
            y_new = spline_interpolation_linear(df[x],num,s)
        elif tipo=='cubic':
            y_new = spline_interpolation_cubic(df[x],num)
        if x==df.columns[0]:
            df_int = pd.DataFrame(data=np.concatenate((y.values.reshape(-1),y_new)),index=indice,columns=[x])
        else :     
            df_n = pd.DataFrame(data=np.concatenate((y.values.reshape(-1),y_new)),index=indice,columns=[x])
            df_int= df_int.join(df_n, how="outer")       
    return df_int


RANDOM SAMPLING:

In [176]:
# Randomly sampling with replacement

def sampling(df,size,freq):
    
    np.random.seed(1)
    indice = series_periodos(df.index[0],size+df.shape[0],freq)
    for x in df.columns:
        data = df[x]
        sampled_data = np.random.choice(data, size=size, replace=True) + np.random.normal(0, 0.5, size)
        if x == df.columns[0]:
            df_sampling=pd.DataFrame(data=np.concatenate((data,sampled_data)),index=indice,columns=[x])
        else:
            df_new = pd.DataFrame(data=np.concatenate((data,sampled_data)),index=indice,columns=[x])
            df_sampling= df_sampling.join(df_new, how="outer")
    return df_sampling

In [177]:
sampling(df,200,'M')

,Valor
2014-01-31,50.362229
2014-02-28,50.237784
2014-03-31,50.813631
2014-04-30,50.862000
2014-05-31,50.862000
...,...
2030-04-30,51.631670
2030-05-31,51.121296
2030-06-30,52.848221
2030-07-31,51.006135


In [178]:
sampling(mult,250,'M')

,Altura,Dinero,Peso
2014-01-31,92.514614,0.695579,107.669842
2014-02-28,91.293370,0.193109,113.707116
2014-03-31,92.876450,-0.000622,109.218068
2014-04-30,92.866858,0.269066,108.237974
2014-05-31,94.905956,0.032981,108.188237
...,...,...,...
2034-06-30,68.422802,-0.006016,110.902469
2034-07-31,91.472775,0.003081,110.393601
2034-08-31,93.058502,0.000093,110.902469
2034-09-30,93.212892,-0.006859,112.489826


TRANSFORMACIÓN MATEMÁTICA:

In [202]:
# Aplicamos log
def agregar_log(df):
    df_o = df.copy()
    for x in df_o.columns:
        df_o[x] = np.log1p(df[x])
    return df_o

In [203]:
# Aplicamos la raíz cuadrada 
def agregar_sqrt(df):
    df_o = df.copy()
    for x in df_o.columns:
        df_o[x] = np.sqrt(df[x])
    return df_o

In [216]:
# Aplicamos la exponencial
def agregar_exp(df,factor):
    df_o = df.copy()
    for x in df_o.columns:
        df_o[x] = np.exp(df_o[x]/factor)
    return df_o 

In [205]:
# Aplicamos el seno

def agregar_sin(df):
    df_o = df.copy()
    for x in df_o.columns:
        df_o[x] = np.sin(df[x])
    return df_o

In [206]:
# Aplicamos el coseno 

def agregar_cos(df):
    df_o = df.copy()
    for x in df_o.columns:
        df_o[x] = np.cos(df_o[x])
    return df_o

In [207]:
# Aplicamos sen + cos

def agregar_trig(df):
    df_o = df.copy()
    for x in df_o.columns:
        df_o[x] = np.cos(df_o[x]) + np.sin(df_o[x])
    return df_o

In [208]:
# Aplicamos sigmoide

def agregar_sigmoid(df):
    df_o = df.copy()
    for x in df.columns:
        df_o[x] = 1 / (1 + np.exp(-df_o[x]))
    return df_o
   

In [213]:
agregar_log(df)

,Valor
indice,
2014-01-31,3.988984
2014-02-28,3.970292
2014-03-31,3.951244
2014-04-30,3.970292
2014-05-31,3.931826
...,...
2023-08-31,3.951244
2023-09-30,3.931826
2023-10-31,3.931826


In [214]:
agregar_log(mult)

,Altura,Dinero,Peso
indice,,,
2014-01-31,4.270471,4.868237e+00,4.735344
2014-02-28,4.239117,4.743632e+00,4.642899
2014-03-31,4.274877,4.487135e+00,4.719572
2014-04-30,4.257960,4.307766e+00,4.697731
2014-05-31,4.483507,4.101992e+00,4.716247
...,...,...,...
2023-08-31,4.549986,1.439682e-08,4.692555
2023-09-30,4.533910,1.041365e-08,4.730976
2023-10-31,4.530686,9.395022e-09,4.709052


In [217]:
agregar_exp(df,2)

,Valor
indice,
2014-01-31,3.227036e+11
2014-02-28,1.957296e+11
2014-03-31,1.187160e+11
2014-04-30,1.957296e+11
2014-05-31,7.200490e+10
...,...
2023-08-31,1.187160e+11
2023-09-30,7.200490e+10
2023-10-31,7.200490e+10


In [218]:
agregar_exp(mult,3)

,Altura,Dinero,Peso
indice,,,
2014-01-31,1.636543e+10,4.874096e+18,2.209724e+16
2014-02-28,7.837503e+09,3.030785e+16,7.732648e+14
2014-03-31,1.818337e+10,5.247815e+12,1.219838e+16
2014-04-30,1.216582e+10,4.050954e+10,5.440599e+15
2014-05-31,4.713901e+12,4.053246e+08,1.077531e+16
...,...,...,...
2023-08-31,3.585026e+13,1.000000e+00,4.504619e+15
2023-09-30,2.167831e+13,1.000000e+00,1.872649e+16
2023-10-31,1.961664e+13,1.000000e+00,8.249812e+15


In [219]:
agregar_cos(df)

,Valor
indice,
2014-01-31,-0.918283
2014-02-28,-0.162991
2014-03-31,0.742154
2014-04-30,-0.162991
2014-05-31,0.964966
...,...
2023-08-31,0.742154
2023-09-30,0.964966
2023-10-31,0.964966


In [220]:
agregar_cos(mult)

,Altura,Dinero,Peso
indice,,,
2014-01-31,0.130153,-0.959352,0.981116
2014-02-28,0.718968,0.729495,-0.676623
2014-03-31,-0.184423,0.995193,-0.395215
2014-04-30,0.852185,-0.525257,-0.307962
2014-05-31,0.913096,-0.973743,-0.702167
...,...,...,...
2023-08-31,0.815812,1.000000,0.250607
2023-09-30,-0.526929,1.000000,0.770488
2023-10-31,-0.754430,1.000000,-0.999961


In [221]:
agregar_sin(df)

,Valor
indice,
2014-01-31,0.395925
2014-02-28,0.986628
2014-03-31,0.670229
2014-04-30,0.986628
2014-05-31,-0.262375
...,...
2023-08-31,0.670229
2023-09-30,-0.262375
2023-10-31,-0.262375


In [222]:
agregar_sin(mult)

,Altura,Dinero,Peso
indice,,,
2014-01-31,0.991494,-2.822130e-01,-0.193421
2014-02-28,-0.695043,6.839865e-01,0.736330
2014-03-31,0.982847,-9.793815e-02,-0.918589
2014-04-30,0.523241,-8.509436e-01,0.951399
2014-05-31,-0.407745,2.276501e-01,-0.712012
...,...,...,...
2023-08-31,-0.578317,1.439682e-08,0.968089
2023-09-30,-0.849909,1.041365e-08,-0.637454
2023-10-31,-0.656380,9.395022e-09,0.008842


In [223]:
agregar_trig(mult)

,Altura,Dinero,Peso
indice,,,
2014-01-31,1.121647,-1.241565,0.787695
2014-02-28,0.023925,1.413481,0.059707
2014-03-31,0.798424,0.897254,-1.313804
2014-04-30,1.375426,-1.376201,0.643437
2014-05-31,0.505350,-0.746093,-1.414179
...,...,...,...
2023-08-31,0.237495,1.000000,1.218696
2023-09-30,-1.376838,1.000000,0.133034
2023-10-31,-1.410811,1.000000,-0.991119


In [224]:
agregar_trig(df)

,Valor
indice,
2014-01-31,-0.522358
2014-02-28,0.823637
2014-03-31,1.412383
2014-04-30,0.823637
2014-05-31,0.702591
...,...
2023-08-31,1.412383
2023-09-30,0.702591
2023-10-31,0.702591


In [225]:
agregar_sigmoid(df)

,Valor
indice,
2014-01-31,1.0
2014-02-28,1.0
2014-03-31,1.0
2014-04-30,1.0
2014-05-31,1.0
...,...
2023-08-31,1.0
2023-09-30,1.0
2023-10-31,1.0


In [226]:
agregar_sigmoid(mult)

,Altura,Dinero,Peso
indice,,,
2014-01-31,1.0,1.0,1.0
2014-02-28,1.0,1.0,1.0
2014-03-31,1.0,1.0,1.0
2014-04-30,1.0,1.0,1.0
2014-05-31,1.0,1.0,1.0
...,...,...,...
2023-08-31,1.0,0.5,1.0
2023-09-30,1.0,0.5,1.0
2023-10-31,1.0,0.5,1.0


In [227]:
agregar_sqrt(df)

,Valor
indice,
2014-01-31,7.280110
2014-02-28,7.211103
2014-03-31,7.141428
2014-04-30,7.211103
2014-05-31,7.071068
...,...
2023-08-31,7.141428
2023-09-30,7.071068
2023-10-31,7.071068


In [228]:
agregar_sqrt(mult)

,Altura,Dinero,Peso
indice,,,
2014-01-31,8.399721,11.361840,10.625568
2014-02-28,8.267198,10.670077,10.141249
2014-03-31,8.418511,9.373713,10.541358
2014-04-30,8.346597,8.560047,10.425833
2014-05-31,9.356528,7.711070,10.523692
...,...,...,...
2023-08-31,9.676317,0.000120,10.398636
2023-09-30,9.598021,0.000102,10.602176
2023-10-31,9.582390,0.000097,10.485557


In [ ]:
# Aplicamos operaciones matemáticas

def agregar_matematica(df,freq,funcion,factor=1):
    
    indice=series_periodos(df.index[0],2*df.shape[0],freq)
    for x in df.columns:
        data = df[x]
        if funcion == 'sqrt':
            transformed_data = np.sqrt(data)
        elif funcion == 'log':
            transformed_data = np.log1p(data)
        elif funcion == 'exp':
            transformed_data = np.exp(data/factor)
        elif funcion == 'sin':
            transformed_data = np.sin(data)
        elif funcion == 'cos':
            transformed_data = np.cos(data)
        elif funcion == 'trig':
            transformed_data = np.cos(data) + np.sin(data)
        elif funcion == 'sigmoide':
            transformed_data = 1 / (1 + np.exp(-data))

        if x == df.columns[0]:
            df_transf=pd.DataFrame(data=np.concatenate((data,transformed_data)),index=indice,columns=[x])
        else:
            df_new = pd.DataFrame(data=np.concatenate((data,transformed_data)),index=indice,columns=[x])
            df_transf= df_transf.join(df_new, how="outer")
    return df_transf

VENTANAS DESLIZANTES:

In [241]:
# Calculo con ventanas deslizantes del tamaño pasado como parámetro

def ventanas(df,ventana):
    df_o = df.copy()
    for x in df.columns:
        df_o[x] = df_o[x].rolling(window=ventana).mean()
    df_o = df_o.dropna()
    return df_o

In [242]:
ventanas(df,4)

,Valor
indice,
2014-04-30,52.00
2014-05-31,51.25
2014-06-30,50.75
2014-07-31,51.00
2014-08-31,50.50
...,...
2023-08-31,50.75
2023-09-30,50.75
2023-10-31,50.50


In [243]:
ventanas(mult,3)

,Altura,Dinero,Peso
indice,,,
2014-03-31,69.924397,1.102695e+02,108.955951
2014-04-30,69.627855,9.166382e+01,107.554387
2014-05-31,76.027206,7.353384e+01,110.188779
2014-06-30,80.837161,5.973296e+01,109.854669
2014-07-31,87.379243,4.945972e+01,110.393227
...,...,...,...
2023-08-31,92.082375,1.724425e-08,109.315692
2023-09-30,92.654452,1.368542e-08,109.778005
2023-10-31,92.525105,1.140183e-08,110.161562


USO DE TECNICAS ESTADÍSTICAS:

In [ ]:
def estadist(df,freq,num,tipo):
    
    indice=series_periodos(df.index[0],num+df.shape[0],freq)
    for x in df.columns:
        data = df[x]
        if tipo==1:
            transformed_data = np.zeros(num)+ data.mean()
        elif tipo==2:
            transformed_data = np.zeros(num) + data.median()
        elif tipo==3:
            transformed_data = np.zeros(num) + data.mode().iloc[0]

        if x == df.columns[0]:
            df_transf=pd.DataFrame(data=np.concatenate((data,transformed_data)),index=indice,columns=[x])
        else:
            df_new = pd.DataFrame(data=np.concatenate((data,transformed_data)),index=indice,columns=[x])
            df_transf= df_transf.join(df_new, how="outer")
            
    return df_transf

In [ ]:
# Devuelve df con datos añadidos calculados a partir de una distribución normal con la media y desviación de los datos pasados 

def normal(df,freq,size):
    
    np.random.seed(1)
    indice=series_periodos(df.index[0],size+df.shape[0],freq)
    for x in df.columns:
        data = df[x]
        mean,std_dev = np.mean(data),np.std(data)
        data_augmented = np.random.normal(mean,std_dev,size=size)
        if x == df.columns[0]:
            df_normal=pd.DataFrame(data=np.concatenate((df[x].values,data_augmented)),index=indice,columns=[x])
        else:
            df_new = pd.DataFrame(data=np.concatenate((data,data_augmented)),index=indice,columns=[x])
            df_normal= df_normal.join(df_new, how="outer")
    return df_normal


In [250]:
normal(df,'M',300)

,Valor
2014-01-31,53.000000
2014-02-28,52.000000
2014-03-31,51.000000
2014-04-30,52.000000
2014-05-31,50.000000
...,...
2048-08-31,48.647194
2048-09-30,51.239068
2048-10-31,51.190669
2048-11-30,50.205917


In [248]:
normal(mult,'M',150)

,Altura,Dinero,Peso
2014-01-31,70.555312,129.091399,112.902687
2014-02-28,68.346558,113.850549,102.844928
2014-03-31,70.871320,87.866499,111.120237
2014-04-30,69.665688,73.274410,108.697997
2014-05-31,87.544611,59.460597,110.748102
...,...,...,...
2036-02-29,91.295795,-6.431972,104.517512
2036-03-31,95.991068,-1.794104,107.419425
2036-04-30,90.193843,11.766259,106.089857
2036-05-31,92.893271,1.436951,111.528189


In [ ]:
# Calcula nuevos datos usando: media + z * desv donde la media y las desv son las de los datos pasados y z = raiz (-2 * log u1) cos(2 pi u2) tal que u1,u2 son dos randoms entre 0 e 1
def box_muller_transform(mean, std_dev, size=100):
    u1, u2 = np.random.rand(size), np.random.rand(size)
    z1 = np.sqrt(-2 * np.log(u1)) * np.cos(2 * np.pi * u2)
    return mean + z1 * std_dev

def box_muller(df,freq,size):
    
    np.random.seed(1)
    indice=series_periodos(df.index[0],size+df.shape[0],freq)
    for x in df.columns:
        data = df[x].values
        data_bm = box_muller_transform(data.mean(),data.std(),size)
        if x == df.columns[0]:
            df_bm=pd.DataFrame(data=np.concatenate((df[x].values,data_bm)),index=indice,columns=[x])
        else:
            df_new = pd.DataFrame(data=np.concatenate((df[x].values,data_bm)),index=indice,columns=[x])
            df_bm = df_bm.join(df_new, how="outer")
    return df_bm

In [254]:
box_muller(df,'M',300)

,Valor
2014-01-31,53.000000
2014-02-28,52.000000
2014-03-31,51.000000
2014-04-30,52.000000
2014-05-31,50.000000
...,...
2048-08-31,49.060947
2048-09-30,50.815040
2048-10-31,51.931063
2048-11-30,49.755833


In [255]:
box_muller(mult,'M',150)

,Altura,Dinero,Peso
2014-01-31,70.555312,129.091399,112.902687
2014-02-28,68.346558,113.850549,102.844928
2014-03-31,70.871320,87.866499,111.120237
2014-04-30,69.665688,73.274410,108.697997
2014-05-31,87.544611,59.460597,110.748102
...,...,...,...
2036-02-29,92.145716,4.262228,113.600218
2036-03-31,88.699016,18.094727,110.894840
2036-04-30,92.505043,6.711823,105.573002
2036-05-31,86.556283,7.446432,108.506238


RUIDO ARMONICO

In [269]:
# Añadimos ruido armonico a la muestra con cierta amplitud y frequencia

def add_harmonic_noise(df,freq,size):
    
    np.random.seed(1)
    df_harm = df.copy()
    for x in df_harm.columns:
        data = df[x]
        time = np.arange(size)
        # Aplicar FFT
        fft_result = np.fft.fft(data)
        frequencies = np.fft.fftfreq(len(data), d=(time[1] - time[0]))  # Frecuencias asociadas
        amplitudes = np.abs(fft_result)  # Magnitudes (amplitud)
        dominant_freq_idx = np.argmax(amplitudes)
        frequency = frequencies[dominant_freq_idx]
        amplitude = amplitudes[dominant_freq_idx]
        harmonic_noise = amplitude * np.sin(2 * np.pi * frequency * time)
        data_augmented = np.random.choice(data, size=size, replace=True) + harmonic_noise
        datos = np.concatenate((data.values,data_augmented))
        if x == df.columns[0]:
            indice = series_periodos(df.index[0],len(datos),freq)
            df_harm = pd.DataFrame(data=datos,index=indice,columns=[x])
        else:
            df_new = pd.DataFrame(data = datos,index=indice,columns=[x])
            df_harm = df_harm.join(df_new, how="outer")
    
    return df_harm

In [272]:
add_harmonic_noise(df,5,0.6)

,Valor
indice,
2014-01-31,52.000000
2014-02-28,47.061074
2014-03-31,57.755283
2014-04-30,47.244717
2014-05-31,53.938926
...,...
2023-08-31,51.000000
2023-09-30,48.061074
2023-10-31,54.755283


In [273]:
add_harmonic_noise(mult,1,0.65)

,Altura,Dinero,Peso
indice,,,
2014-01-31,70.555312,129.091399,112.902687
2014-02-28,67.537541,113.041532,102.035911
2014-03-31,71.822376,88.817555,112.071294
2014-04-30,69.356671,72.965393,108.388980
2014-05-31,86.956825,58.872811,110.160317
...,...,...,...
2023-08-31,92.631116,-1.000000,107.131639
2023-09-30,92.709786,0.587785,112.993931
2023-10-31,92.131215,0.309017,110.255917


DUPLICADO + PERTURBACIÓN:

In [102]:
# Duplicar algunos datos y añadir ruido
def duplicate_and_perturb(data, duplication_factor=0.3, perturbation_std=0.05):
    duplicated_data = []
    np.random.seed(8)
    for point in data:
        duplicated_data.append(point)
        if np.random.rand() < duplication_factor:
            duplicated_data.append(point + np.random.normal(0, perturbation_std))
    return np.array(duplicated_data)

In [285]:
# Duplicamos algunos datos añadiendole cierto ruido.

def duplicados(df,freq,duplication_factor=0.3,perturbation_std=0.05):
    
    np.random.seed(1)
    for x in df.columns:
        data = df[x]
        data_dd=duplicate_and_perturb(data,duplication_factor,perturbation_std)
        if x == df.columns[0]:
            indice = series_periodos(df.index[0],len(data_dd),freq)
            df_dd = pd.DataFrame(data=data_dd,index=indice,columns=[x])
        else:
            df_new = pd.DataFrame(data = data_dd,index=indice,columns=[x])
            df_dd = df_dd.join(df_new, how="outer")
            
    return df_dd

In [286]:
duplicados(df,'M')

,Valor
2014-01-31,52.015530
2014-02-28,50.015530
2014-03-31,53.015530
2014-04-30,52.015530
2014-05-31,51.015530
...,...
2026-05-31,50.955510
2026-06-30,49.994629
2026-07-31,49.874868
2026-08-31,49.994629


In [287]:
duplicados(mult,'M')

,Altura,Dinero,Peso
2014-01-31,70.552601,129.105362,112.914422
2014-02-28,68.343848,113.864512,102.856663
2014-03-31,70.868610,87.880462,111.131972
2014-04-30,69.662978,73.288373,108.709731
2014-05-31,87.541900,59.474560,110.759837
...,...,...,...
2026-05-31,92.066828,-0.043333,112.377499
2026-06-30,91.806144,-0.004215,109.957372
2026-07-31,91.686384,-0.123976,109.837611
2026-08-31,92.304843,-0.004215,109.462807


COMBINACIÓN LINEAL:

In [110]:
# Calculamos nuevos datos como combinación lineal de los otros 
def linear_combinations(data,num_datos, n_combinations):
    for _ in range(num_datos):
        datos = data[-n_combinations:]
        weights = np.random.rand(n_combinations)
        weights /= np.sum(weights)  # Normalizar pesos
        combination = np.dot(weights, datos)
        combination += np.random.normal(0,0.5)
        data=np.append(data,combination)
    return np.array(data)

In [294]:
def agregar_comb(df,freq,size,window_size):
    
    np.random.seed(1)
    for x in df.columns:
        data = df[x]
        datos = linear_combinations(data.values,size,window_size)
        if x == df.columns[0]:
            indice = series_periodos(df.index[0],len(datos),freq)
            df_dl = pd.DataFrame(data=datos,index=indice,columns=[x])
        else:
            df_new = pd.DataFrame(data = datos,index=indice,columns=[x])
            df_dl = df_dl.join(df_new, how="outer")
    return df_dl


In [295]:
agregar_comb(df,'M',1000,5)

,Valor
2014-01-31,52.015530
2014-02-28,50.015530
2014-03-31,53.015530
2014-04-30,52.015530
2014-05-31,51.015530
...,...
2106-12-31,50.836750
2107-01-31,50.796728
2107-02-28,50.787233
2107-03-31,50.826144


In [296]:
agregar_comb(mult,'M',200,5)

,Altura,Dinero,Peso
2014-01-31,70.552601,129.105362,112.914422
2014-02-28,68.343848,113.864512,102.856663
2014-03-31,70.868610,87.880462,111.131972
2014-04-30,69.662978,73.288373,108.709731
2014-05-31,87.541900,59.474560,110.759837
...,...,...,...
2040-04-30,92.444450,5.773568,109.653305
2040-05-31,92.553942,5.147852,109.457663
2040-06-30,92.673837,6.861715,109.542528
2040-07-31,92.688295,6.966874,109.368821


DESCOMPOSICIÓN

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
def descomp(df,size,freq,tipo):
    
    indice=series_periodos(df.index[0],size+df.shape[0],freq)

    for x in df.columns:
        data = df[x]
        # Descomposición de la serie
        if tipo=="additive":
            descomposicion = seasonal_decompose(data, model='additive', period=12)
        elif tipo=="multiplicative":
            descomposicion = seasonal_decompose(data, model='multiplicative', period=12)
            
        tendencia = descomposicion.trend
        estacionalidad = descomposicion.seasonal
        residuo = descomposicion.resid
        # Calcular la tasa de cambio promedio de la tendencia
        tendencia_valida = tendencia.dropna()
        cambios = tendencia_valida.diff().dropna()
        tasa_cambio_promedio = cambios.mean()

        # Extrapolar los valores de la tendencia
        n_pasos = size
        ultima_tendencia = tendencia_valida.iloc[-1]
        tendencia_futura = [ultima_tendencia + (i + 1) * tasa_cambio_promedio for i in range(n_pasos)]
        
        # Replicar los valores estacionales
        longitud_estacionalidad = 12  # Basado en la periodicidad detectada
        estacionalidad_extrapolada = np.tile(estacionalidad[-longitud_estacionalidad:], int(size/12)+1)[:size]
        if tipo=="additive":
            prediccion = tendencia_futura + estacionalidad_extrapolada
        elif tipo=="multiplicative":
            prediccion = tendencia_futura * estacionalidad_extrapolada
        if x == df.columns[0]:
            df_desc=pd.DataFrame(data=np.concatenate((data,prediccion)),index=indice,columns=[x])
        else:
            df_new = pd.DataFrame(data=np.concatenate((data,prediccion)),index=indice,columns=[x])
            df_desc= df_desc.join(df_new, how="outer")
    return df_desc

MODELOS DE PREDICCIÓN:

In [ ]:
from skforecast.Sarimax import Sarimax
from skforecast.ForecasterSarimax import ForecasterSarimax
from skforecast.model_selection_sarimax import backtesting_sarimax
from skforecast.model_selection_sarimax import grid_search_sarimax
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from skforecast.model_selection import grid_search_forecaster
from sklearn.metrics import mean_squared_error
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from sklearn.linear_model import Ridge
from prophet import Prophet
from sklearn.preprocessing import StandardScaler

In [ ]:
# Definición de modelo autorregresivos con búsqueda de parámetros realizada por grid search devolviendo la predicción

def prediccion_sarimax(datos,datos_train, columna):
    
    # Grid search
    forecaster = ForecasterSarimax(
                    regressor=Sarimax(
                                    order=(1, 1, 1), # Placeholder replaced in the grid search
                                    maxiter=500
                                )
                )

    param_grid = {
        'order': [(0, 1, 0), (0, 1, 1), (1, 1, 0), (1, 1, 1), (2, 1, 1), (1 ,1 ,2), ( 2, 1, 2),(0, 0, 0), (0, 0, 1), (1, 0, 0), (1, 0, 1), (2, 0, 1), (1 ,0 ,2), ( 2, 0, 2) ],
        'seasonal_order': [(0, 0, 0, 0), (0, 1, 0, 12), (1, 1, 1, 12)],
        'trend': [None]
    }

    resultados_grid = grid_search_sarimax(
                            forecaster            = forecaster,
                            y                     = datos[columna],
                            param_grid            = param_grid,
                            steps                 = 12,
                            refit                 = True,
                            metric                = 'mean_absolute_error',
                            initial_train_size    = int(len(datos_train)*0.8),
                            fixed_train_size      = False,
                            return_best           = False,
                            n_jobs                = 'auto',
                            suppress_warnings_fit = True,
                            verbose               = False,
                            show_progress         = True
                    )
    
    r=resultados_grid.index[0]

    # Predicciones de backtesting con el mejor modelo según el grid search
    # ==============================================================================
    forecaster_1 = ForecasterSarimax( regressor=Sarimax(order=resultados_grid.order[r], seasonal_order=resultados_grid.seasonal_order[r], maxiter=500),
                    )

    metrica_m1, predicciones_m1 = backtesting_sarimax(
                                            forecaster            = forecaster_1,
                                            y                     = datos[columna],
                                            initial_train_size    = int(len(datos_train)*0.8),
                                            steps                 = 72,
                                            metric                = 'mean_absolute_error',
                                            refit                 = True,
                                            n_jobs                = "auto",
                                            suppress_warnings_fit = True,
                                            verbose               = False,
                                            show_progress         = True
                                        )

    
    return predicciones_m1


In [ ]:

def prediccion_backtesting_forecasterAutoreg(datos_train,column,size,lags,steps):

    forecaster = ForecasterAutoreg(
                    regressor = RandomForestRegressor(random_state=123),
                    lags      = lags
                )
    # Valores candidatos de lags
    lags_grid = [10, 20]

    # Valores candidatos de hiperparámetros del regresor
    param_grid = {
         'n_estimators': [100, 175,250,450],
         'max_depth': [3, 5, 10,15,20]
    }

    resultados_grid = grid_search_forecaster(
                        forecaster         = forecaster,
                        y                  = datos_train[column],
                        param_grid         = param_grid,
                        lags_grid          = lags_grid,
                        steps              = steps,
                        refit              = False,
                        metric             = 'mean_squared_error',
                        initial_train_size = int(len(datos_train)*0.8),
                        fixed_train_size   = False,
                        return_best        = True,
                        n_jobs             = 'auto',
                        verbose            = False
                    )

    # Predicciones
    # ==============================================================================
    predicciones = forecaster.predict(steps=size)

    return predicciones

In [ ]:

def predicciones_backtesting_forecasterAutoregDirect(datos_train,column,steps,lags):

    forecaster = ForecasterAutoregDirect(
                regressor     = Ridge(random_state=123),
                transformer_y = StandardScaler(),
                steps         = steps,
                lags          = lags
             )

    # Valores candidatos de lags
    lags_grid = [5, 12, 20]

    # Valores candidatos de hiperparámetros del regresor
    param_grid = {'alpha': np.logspace(-5, 5, 10)}

    resultados_grid = grid_search_forecaster(
                        forecaster         = forecaster,
                        y                  = datos_train[column],
                        param_grid         = param_grid,
                        lags_grid          = lags_grid,
                        steps              = steps,
                        refit              = False,
                        metric             = 'mean_squared_error',
                        initial_train_size = int(len(datos_train)*0.8),
                        fixed_train_size   = False,
                        return_best        = True,
                        n_jobs             = 'auto',
                        verbose            = False
                    )

    # Predicciones
    # ==============================================================================
    predicciones = forecaster.predict()

    # Error de test
    # ==============================================================================
    return predicciones


In [ ]:
# Definimos el modelo de predicción prophet cuyos parámetros son unos datos de entrenamiento y otros de test y devolvemos las predicciones

def pred_prophet_prediccion(data_train,column,size,frequ):
    
    data_train=data_train.reset_index()
    data_train.rename(columns={data_train.columns[0] : 'ds', column: 'y'}, inplace=True)
    model = Prophet()
    model.fit(data_train)
    
    future = model.make_future_dataframe(periods=size,freq=frequ)
    forecast=model.predict(future)
    
    y_pred=forecast['yhat'][len(data_train):].values
    
    return y_pred
